# Course Timetabling: Case Study for the University Course Timetabling Problem

## Phase 1: Allocation of Professors to Courses

Objective:

- Maximize academic performance (EAP, professor qualified to teach the course) and the number of weak constraints met
- Allocate permanent and substitute professors to courses (mandatory, elective, and service) according to the rules of the Institute of Computing at UFRJ

## Model Formulation

### Notation

- **$TObg$**: Set of classes related to mandatory courses 
- **$TOpt$**: Set of classes related to elective courses 
- **$TSvc$**: Set of classes related to service courses 
- **$T$**: Set of classes to be offered 
- **$PE$**: Set of tenured professors  
- **$PS$**: Set of substitute professors  
- **$P$**: Set of all professors: tenured and substitute, also including the DUMMY professor 
- **$TQ_p$**: Subset of classes whose course can be taught by professor $p$ 
- **$H_{tid}$**: Set of time slots in which class $t$ is offered 
- **$D_{tid}$**: Set of weekdays in which class $t$ is offered 
- **$T_h$**: Set of classes that have the same time slot 
- **$T_d$**: Set of classes that occur on the same day 
- **$AM_{p,t}$**: Manually assigned set that allocates a professor $p$ to a class $t$ 


### Variables

Considering

$$
    p \in P, t \in T, tid = \pi_{id}(t), (d,h) \in TS_{tid}
$$

1. Decision variable $X_{p,tid,d,h}$ that determines the allocation of a professor to a class:

$$
  X_{p,tid,d,h}  =
    \begin{cases}
    1 &  \quad \text{if professor $p$ is allocated to class $t$ for the timeslot $(d,h) \in TS_{tid}$}\\ 
    0 & \quad \text{otherwise} 
    \end{cases} 
$$

2. Slack variable $ECH^{-}_{p}$ for the permanent professor: indicates how many credits the permanent professor is below the ideal set by the coordination:

$$
ECH^{-}_{p} \quad \text{where } p \in PE \wedge ECH^{-}_{p} \in [0, MinChPE]
$$

3. Slack variable $SCH^{-}_{p}$ for the substitute professor: indicates how many credits the substitute professor is below the ideal set by the coordination:

$$
SCH^{-}_{p} \quad \text{where } p \in PS \wedge SCH^{-}_{p} \in [0, MinChPS]
$$


### Coefficients

The coefficients are values that act as weights on the variables in the objective function. The choice of these weights was obtained through experimentation and followed some basic principles: the DUMMY professor should be avoided, being used only when there is no other professor available to teach a course, in order not to make the model unfeasible. Therefore, its default value is close to zero $(0.0001)$. The weight of the professors follows a power of 10, so that substitute professors teaching service courses are generally preferred over permanent professors. In addition, the penalty for the professor not reaching the minimum workload is an institutional requirement. 
Let's look at the details of each scenario:


1. If the class $(t)$ is associated with a mandatory course $(t \in TObg)$ and the professor $(p)$ is qualified to teach the course $(TQ_p)$ or is in the manual allocation $(AM_{p,t})$, the weight will be 100:

$$
EAPI_{p,t} = 
\begin{cases}
100 &  \quad t \in (TObg \cap (TQ_p \cup AM_{p,t})) \wedge p \neq \text{DUMMY}\\ 
0 & \quad \text{otherwise}
\end{cases}
$$


2. If the class $(t)$ is associated with a service course $(t \in TSvc)$ that is basic, all professors have prior knowledge to teach this course. However, normally in the institute, substitute professors $(p \in PS)$ teach this course. For permanent professors $(p \in PE)$, the weight will be lower:

$$
EAPI_{p,t}  = 
\begin{cases}
10 &  \quad t \in TSvc \wedge p \in PS\\ 
1 &  \quad t \in TSvc \wedge p \in PE\\ 
0 & \quad \text{otherwise}
\end{cases}
$$


3. The DUMMY professor is qualified in all courses. He receives a coefficient of 0.0001 because he should only be considered if there is no other qualified professor. Thus, we do not make the model unfeasible in case we do not have a professor to be allocated:

$$
EAPI_{\text{DUMMY},t} = 0.0001
$$

> The factor \( 0.0001 \) is adopted according to the literature (TRIGOS;CORONEL,2023) as it is considered a negligible value. Thus, the preference will always be to allocate a professor from the university's available staff.  

4. Penalty factor in case the permanent professor does not reach the minimum required credits:

$$
    Wf_{pe} = 100 \quad pe \in PE
$$

5. Penalty factor in case the substitute professor does not reach the minimum required credits:

$$
    Wf_{ps} = 1000 \quad ps \in PS
$$

### Objective Function

Maximize the expected academic performance, \textit{Expected Academic Performance} (EAP), and the number of weak constraints met, penalizing cases where the permanent professor (slack variable $ECH^{-}_p$) and the substitute professor (slack variable $SCH^{-}_p$) do not reach the required workload. The corresponding equation for this case is given by equation bellow, where, for each professor who does not meet the minimum load, there is a decrease in the value of the objective function. Since the function involves a maximization problem, whenever possible, this weak constraint will be met.

$$
    MAX \sum_{p \in P}\sum_{t \in T}\sum_{(d,h) \in TS_{tid}} EAP_{p,t}X_{p,tid,d,h} - Wf_{pe} \sum_{p \in PE} ECH^{-}_{p} - Wf_{ps} \sum_{p \in PS} SCH^{-}_{p}
$$

## Constraints
### Weak Constraints

**RNG1 (General Business Rule 1)**: Ensures that the permanent professor $(p \in PE)$ is allocated with the amount of credits suggested by the coordination ($MinChPE$) if possible. Does not make the model unfeasible if not achieved. 

$$
    \sum_{t \in T} Cred_{tid} X_{p,tid,d,h} = MinChPE - ECH^{-}_{p} \quad p \in PE \wedge tid = \pi_{id}(t)
$$

> The logic of function describes that for each fixed permanent professor, all options of associated courses are checked.

**RNG2**: Ensures that the substitute professor $(p \in PS)$ is allocated with the amount of credits suggested by the coordination ($MinChPS$) if possible. Does not make the model unfeasible if not achieved. 

$$
    \sum_{t \in T} Cred_{tid} X_{p,tid,d,h} = MinChPS - SCH^{-}_{p}, \quad p \in PS \wedge tid = \pi_{id}(t)
$$

The logic of function describes that for each fixed substitute professor, all options of associated courses are checked.

### Hard Constraints

**RNG3**: A class $(t)$ must be taught by a single professor $(p)$.

$$
    \sum_{p \in P}\sum_{t \in T} X_{p,tid,d,h} = 1,  \quad (d,h) \in TS_{tid} \wedge tid = \pi_{id}(t)
$$

**RNG4**: A professor $(p)$ can only teach 1 class $(t)$ on the same day $(d)$ and time $(h)$. Does not involve the DUMMY professor.

$$
    \sum_{p \in (PE \cup PS)}\sum_{tid \in (T_h \cap T_{d})} X_{p,tid,d,h} \leq 1,  \quad (d,h) \in TS_{tid} \wedge tid = \pi_{id}(t)
$$

> Equation considers all substitute and permanent professors. The sets \( T_d \), and \( T_h \), represent, respectively, all classes that have courses on the same day and time.  

**RNG5**: A professor $(p)$ cannot teach a course in which he is not qualified $(tid \notin TQ_{p}, \text{ where } tid = \pi_{id}(t))$. Let \(\Omega\) be the universe set of all possible class identifiers ($tid$) associated with the courses, the corresponding equation is given by:

$$
    \sum_{p \in P} \sum_{tid \in (\Omega \setminus (TQ_{p} \cup \Pi_{tid}(AM_{p,t}))} X_{p,tid,d,h} = 0, \quad  (d,h) \in TS_{tid} \wedge tid = \pi_{id}(t)
$$

> If there is manual allocation in the model input, we will not have validation of the professor's qualification associated with the course.

**RNP1 (Particular Business Rule 1)**: Manual allocations $(AM_{p,t})$ must be allocated mandatorily. 

$$
    \begin{aligned}
        \sum_{(p, t) \in AM_{p,t}} X_{p,tid,d,h} = 1, \quad  
         (p, t) \in AM_{p,t} \wedge 
        tid = \pi_{id}(t) \wedge  
        (d,h) \in TS_{tid}
    \end{aligned}
$$

> The logic of equation establishes that, for each manual instance in which a professor (p) teaches a class (t) provided as input, the corresponding allocation must be present in the model solution.

**RNP2**: Workload regime associated with the permanent professor ($p \in PE$), where the maximum amount of credits is defined ($MaxChPE$):

$$
    \sum_{t \in T} Cred_{tid} X_{p,tid,d,h} \leq MaxChPE \quad p \in PE \wedge tid = \pi_{id}(t)
$$

> The logic of function describes that for each fixed professor, all options of associated courses are checked.

**RNP3**: Workload regime associated with the substitute professor ($p \in PS$), where the maximum amount of credits is defined ($MaxChPS$):

$$
    \sum_{t \in T} Cred_{tid} X_{p,tid,d,h} \leq MaxChPS \quad p \in PS \wedge tid = \pi_{id}(t)
$$

> The logic of function describes that for each fixed professor, all options of associated courses are checked.

### Final Formulation

By unifying the formulation, we obtain the mathematical modeling expressed by equation:

$$
\begin{equation}
\begin{aligned}
    & \text{MAX} \quad \sum_{p \in P}\sum_{t \in T} \sum_{(d,h) \in TS_{tid}} EAP_{p,t}X_{p,tid,d,h} - Wf_{pe} \sum_{p \in PE} ECH^{-}_{p} - Wf_{ps} \sum_{p \in PS} SCH^{-}_{p} \\ \\
    & \text{Subject to:} \\
    & \quad \sum_{t \in T} \text{Cred}_{tid} X_{p,tid,d,h} = MinChPE - ECH^{-}_{p}, \quad \forall p \in PE \\
    & \quad \sum_{t \in T} \text{Cred}_{tid} X_{p,tid,d,h} = MinChPS - SCH^{-}_{p} \quad \forall p \in PS \\
    & \quad \sum_{t \in T} \text{Cred}_{tid} X_{p,tid,d,h} \leq MaxChPS, \quad \forall p \in PS \\
    & \quad \sum_{t \in T} \text{Cred}_{tid} X_{p,tid,d,h} \leq MaxChPE, \quad \forall p \in PE \\
    & \quad \sum_{(p, tid, dis) \in AM_{p,t}} X_{p,tid,d,h} = 1, \quad \forall (p,tid, dis) \in AM_{p,t}  \wedge p \neq \text{DUMMY} \\
    & \quad \sum_{p \in P}\sum_{t \in T} X_{p,tid,d,h} = 1, \quad \forall (d,h) \in TS_{tid} \\
    & \quad \sum_{p \in (PE \cup PS)}\sum_{tid \in (T_h \cap T_{d})} X_{p,tid,d,h} \leq 1, \quad \forall (d,h) \in TS_{tid} \\
    & \quad \sum_{p \in P} \sum_{t \in (\Omega \setminus TQ_{p})} X_{p,tid,d,h} = 0, \quad \forall (d,h) \in TS_{tid} \text{ such that } (p, tid, dis) \notin AM_{p,t}\\
    & \quad EAP_{p,t} = EAPI_{p,t} = 
    \begin{cases}
        100 & \text{if } t \in (TQ_p \cup AM_{p,t}) \wedge p \neq \text{DUMMY} \\ 
        10 & \text{if } t \in TSvc \wedge p \in PS \\ 
        1 & \text{if } t \in TSvc \wedge p \in PE \\ 
        0.0001 & \text{if } p = \text{DUMMY} \\ 
        0 & \text{otherwise}
    \end{cases} \\
    & \quad X_{p,tid,d,h} \in \{0, 1\}, \quad \forall p \in P, t \in T, (d,h) \in TS_{tid}, tid = \pi_{id}(t)
\end{aligned}
\end{equation}
$$